In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2

Extract oriented bounding box (individual tooth) from the radiograph

In [ ]:
def get_sub_image(image, x, y, width, height, rotation):
    """Extracts & transforms the rotated bounding box of the input image to a vertical subimage."""
    rotation *= -1  # Reverse the rotation direction
    original_height, original_width = image.shape[:2] # Get the image's original width and height
    
    # Inputs x (xmin), y (ymin), width, height are in percentage values
    # Convert the percentage values to pixel values
    pixel_x = x / 100.0 * original_width
    pixel_y = y / 100.0 * original_height
    pixel_width = width / 100.0 * original_width
    pixel_height = height / 100.0 * original_height

    # Calculate the center of the bounding box
    center_x = pixel_x + pixel_width / 2
    center_y = pixel_y + pixel_height / 2

    rotation_matrix = np.array([
        [np.cos(np.radians(rotation)), -np.sin(np.radians(rotation)), 0],
        [np.sin(np.radians(rotation)), np.cos(np.radians(rotation)), 0],
        [0, 0, 1]
    ])

    # Create the translation matrices
    translation_matrix_to_origin = np.array([
        [1, 0, -center_x],
        [0, 1, -center_y],
        [0, 0, 1]
    ])
    translation_matrix_back = np.array([
        [1, 0, pixel_width / 2],
        [0, 1, pixel_height / 2],
        [0, 0, 1]
    ])
    
    # Calculate the affine matrix
    affine_matrix = np.dot(rotation_matrix, translation_matrix_to_origin)
    affine_matrix = np.dot(affine_matrix, translation_matrix_back)
    # print(affine_matrix.dtype)
    # print(affine_matrix)

    rotated = cv2.warpPerspective(
        image, affine_matrix, (int(pixel_width), int(pixel_height)))


    # Display the subimage
    print(rotated.shape)
    plt.imshow(cv2.cvtColor(rotated, cv2.COLOR_BGR2RGB))
    plt.show()
    plt.imshow(cv2.cvtColor(rotated, cv2.COLOR_BGR2GRAY), cmap='gray')
    plt.axis('off')
    plt.show()

Test on some train samples

In [ ]:
file = pd.read_csv('train_annotations.csv', keep_default_na=False)
for index, row in file.iterrows():
    image_path = row['image'].split('_')[0] +'/'+ row['image']
    image = cv2.imread(image_path)
    if image is None:
        continue
    get_sub_image(image, row['x'], row['y'], row['width'], row['height'], row['rotation'])
    if index == 5:
        break